Description: stiches sensors and potholes files together into a new combined file using a given time window

In [27]:
import pandas as pd
pd.options.display.max_rows = 100
pd.set_option('display.precision', 10)  #for displaying timestamps

In [56]:
sensorsFilePath = 'data/Route Brainstorming 419/r5_sensors.csv'
potholesFilePath = 'data/Route Brainstorming 419/r5_potholes.csv'
combinedFilePath = 'data/Route Brainstorming 419/r5_combined.csv'
sensorsDF = pd.read_csv(sensorsFilePath)
potholesDF = pd.read_csv(potholesFilePath)
window = 1.0   #pothole time window in seconds

In [57]:
sensorsDF.head()

,timestamp,latitude,longitude,speed,accelerometerX,accelerometerY,accelerometerZ,gyroX,gyroY,gyroZ
0,1.4926384067e+09,40.4482732887,-79.9456688489,2.2400000095,0.0171966553,-0.9292755127,0.2934722900,-0.0245407631,-0.0126055404,-0.0034173683
1,1.4926384069e+09,40.4482732887,-79.9456688489,2.2400000095,0.0570983887,-0.9299468994,0.2604522705,-0.0306681642,-0.0051220577,-0.0184264116
2,1.4926384071e+09,40.4482732887,-79.9456688489,2.2400000095,0.0630187988,-0.9326782227,0.2527923584,-0.0302372647,-0.0466061180,0.0051417651
3,1.4926384073e+09,40.4482881972,-79.9456919537,2.5199999809,0.0386962891,-0.9516754150,0.3287048340,0.0034085799,0.0031667649,-0.0138340566
4,1.4926384075e+09,40.4482881972,-79.9456919537,2.5199999809,0.0514526367,-0.9450073242,0.3589324951,-0.0374040975,-0.0220951823,-0.0024583640


In [58]:
potholesDF

,timestamp
0,1.4926385337e+09
1,1.4926385404e+09
2,1.4926386127e+09
3,1.4926386133e+09
4,1.4926386196e+09
5,1.4926386227e+09
6,1.4926386245e+09
7,1.4926386259e+09
8,1.4926388072e+09
9,1.4926388104e+09


In [59]:
potholeLabels = []   #indicator for pothole at each timestamp
sensorIndex = 0
sensorTimestamp = sensorsDF['timestamp'][sensorIndex]
halfWindow = window/2

#Adds pothole indicators based on time and window
for index, potholeTimestamp in potholesDF['timestamp'].iteritems():
    while sensorTimestamp <= (potholeTimestamp + halfWindow):
        if (potholeTimestamp - halfWindow) <= sensorTimestamp:
            potholeLabels.append(True)
        else:
            potholeLabels.append(False)
        sensorIndex += 1
        sensorTimestamp = sensorsDF['timestamp'][sensorIndex]

#No potholes for remainder of data 
#(when sensorsDF larger than potholesDF)
remainingIndices = len(sensorsDF) - sensorIndex
remainingPotholeLabels = [False] * remainingIndices
potholeLabels.extend(remainingPotholeLabels)

In [60]:
sensorsDF['potholes'] = potholeLabels
sensorsDF.head()

,timestamp,latitude,longitude,speed,accelerometerX,accelerometerY,accelerometerZ,gyroX,gyroY,gyroZ,potholes
0,1.4926384067e+09,40.4482732887,-79.9456688489,2.2400000095,0.0171966553,-0.9292755127,0.2934722900,-0.0245407631,-0.0126055404,-0.0034173683,False
1,1.4926384069e+09,40.4482732887,-79.9456688489,2.2400000095,0.0570983887,-0.9299468994,0.2604522705,-0.0306681642,-0.0051220577,-0.0184264116,False
2,1.4926384071e+09,40.4482732887,-79.9456688489,2.2400000095,0.0630187988,-0.9326782227,0.2527923584,-0.0302372647,-0.0466061180,0.0051417651,False
3,1.4926384073e+09,40.4482881972,-79.9456919537,2.5199999809,0.0386962891,-0.9516754150,0.3287048340,0.0034085799,0.0031667649,-0.0138340566,False
4,1.4926384075e+09,40.4482881972,-79.9456919537,2.5199999809,0.0514526367,-0.9450073242,0.3589324951,-0.0374040975,-0.0220951823,-0.0024583640,False


In [61]:
sensorsDF[sensorsDF['potholes'] == True]

,timestamp,latitude,longitude,speed,accelerometerX,accelerometerY,accelerometerZ,gyroX,gyroY,gyroZ,potholes
633,1.4926385333e+09,40.4532008645,-79.9475675945,13.1199998856,-0.5370025635,-0.0936584473,0.5059967041,-0.3374621912,0.1106902960,-0.0261312029,True
634,1.4926385335e+09,40.4532008645,-79.9475675945,13.1199998856,-0.8103942871,-0.6601715088,0.2329254150,-0.0362818414,-0.0678221908,0.1234843882,True
635,1.4926385337e+09,40.4532008645,-79.9475675945,13.1199998856,0.1088256836,-0.8430786133,0.3420104980,-0.0652442509,0.0078744347,0.0249008225,True
636,1.4926385339e+09,40.4532008645,-79.9475675945,13.1199998856,0.2330780029,-1.1494445801,0.0478057861,0.0985390909,0.1759326827,-0.0229018538,True
637,1.4926385341e+09,40.4532008645,-79.9475675945,13.1199998856,0.0098571777,-1.0866851807,0.1109619141,-0.0143376604,0.0543410031,-0.0120153839,True
666,1.4926385399e+09,40.4529473142,-79.9484422769,12.7200002670,-0.0633239746,-1.0214233398,0.2042694092,-0.0961808617,0.0229572475,-0.0404938970,True
667,1.4926385401e+09,40.4529473142,-79.9484422769,12.7200002670,0.1316375732,-1.1787109375,0.1704864502,0.2490601567,-0.0282015444,0.0719266547,True
668,1.4926385403e+09,40.4529073260,-79.9485852647,12.6999998093,0.1618347168,-0.3713531494,-0.0428161621,0.3605462052,0.0540046458,-0.1691746451,True
669,1.4926385405e+09,40.4529073260,-79.9485852647,12.6999998093,0.1467132568,-1.1316833496,0.2101287842,-0.0566430395,-0.0155738997,0.0836911688,True
670,1.4926385407e+09,40.4529073260,-79.9485852647,12.6999998093,0.0390319824,-0.7686614990,0.2051849365,-0.2947349673,-0.0129789156,-0.0253745989,True


In [62]:
sensorsDF.to_csv(combinedFilePath)